# ChatGPT API
Using ChatGPT API for various reasons - summaries, theme annotation, etc

In [2]:
import sys, os
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from ast import literal_eval
from speach import elan
from difflib import SequenceMatcher
import re
import json
import time
from tqdm import tqdm

import IPython
import ipywidgets as widgets
from IPython.display import Markdown, display

In [3]:
# Loading and parametrizing
import openai
import tiktoken # get the number of tokens: use tiktoken

with open("../src/openai_apk", 'r') as f:
    d = {x.split('=')[0]:literal_eval(x.split('=')[1]) for x in f.readlines()}
openai.api_key = d['API_KEY']

## Functions

### ChatGPT prompt / calls

In [4]:
def get_number_tokens(s:str, model:str="gpt-3.5-turbo"):
    enc = tiktoken.get_encoding("cl100k_base") # can use for other models
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    encs = enc.encode(s)
    return len(encs)

In [5]:
def create_prompt(s:str, speakers:list):
    s = f"""ce texte est extrait d'une conversation entre {' et '.join(speakers)}. lorsqu'un nouveau locuteur prend la parole, son nom est indiqué entre crochets (par exemple <{speakers[0]}>). Peux-tu me donner les différents thèmes de la conversation et me citer la phrase par laquelle ils débutent ? par exemple: "thème 1: [thème] (phrase d'introduction: [extrait du texte])"

conversation: "{s}" """
    return s

In [294]:
def create_prompt_solution(conversation:str, speakers:list):
    speaker = speakers[0]
    return f"""Dans la conversation suivante, les locuteurs (indiqués entre crochets, par exemple <{speaker}>) cherchent à résoudre un dilemme moral : ils doivent choisir qui, parmi les passager d'une montgolfière, jeter par dessus bord pour sauver les autres. Que choisissent-ils ? . Répond en un mot parmi les propositions suivantes: "scientifique", "maitresse", "pilote", "aucun".

conversation: \"{conversation}\""""

def create_prompt_group_themes(themes, mode:str='group'):
    """
    themes arg: flatten themes found in parse_response; function not called if 1 answer is not parsed properly
        [y for x in calls for y in parse_chatgpt_res(calls[x]['response'], 'all')[0]]
    """
    auth_modes = ['group','dilemma']
    if mode not in auth_modes:
        raise ValueError(f"Authorised modes are: {auth_modes}; currently {mode}")
    if mode == "group":
        prompt = """A partir de la liste des sujets ci-après, peux-tu me créer des groupes de sujets adjacents similaires ? Si deux sujets similaires sont séparés par un autre thème, alors le sujet au milieu appartient au même groupe que les deux autres (par exemple, si les sujets 1 et 3 appartiennent au même groupe, alors le thème 2 est dans le même groupe que les thèmes 1 et 3).
        
Les thèmes sont:\n"""
    elif mode == "dilemma":
        prompt = """Parmi les thèmes ci-après, lesquels sont liés à la résolution du dilemme moral du choix d'un passager d'une montgolfière à sacrifier ?
    
Les thèmes sont:\n"""
    for i, theme in enumerate(themes):
        prompt += f"{i+1}. {theme}\n"
    return prompt

In [6]:
def call_api(s:str, model:str="gpt-3.5-turbo"):#, max_tokens:int=256):
    """
    Response:
    {
        'id': 'chatcmpl-6p9XYPYSTTRi0xEviKjjilqrWU2Ve',
        'object': 'chat.completion',
        'created': 1677649420,
        'model': 'gpt-3.5-turbo',
        'usage': {'prompt_tokens': 56, 'completion_tokens': 31, 'total_tokens': 87},
        'choices': [
        {
            'message': {
            'role': 'assistant',
            'content': 'The 2020 World Series was played in Arlington, Texas at the Globe Life Field, which was the new home stadium for the Texas Rangers.'},
            'finish_reason': 'stop', # 'length' (token limit), 'content_filter' (flagged), 'null' (still in progress)
            'index': 0
        }
        ]
    }
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ {"role": "user", "content": s} ],
# Parametrizing not supported
#        temperature=0.7,
#        max_tokens=max_tokens,
#        top_p=1,
#        frequency_penalty=0,
#        presence_penalty=0
    )
    res = response['choices'][0]['message']['content']
    nb_tok = {'total': response['usage']['total_tokens'], 'response': response['usage']['completion_tokens']}
    return res, nb_tok

### Prepare text for call

In [7]:
transcr_loc = "../data/transcript/current"
transcr_files = sorted([x for x in os.listdir(transcr_loc) if 'eaf' in x])
markers_path = "../data/video/markers_from_video_start.csv"
markers = pd.read_csv(markers_path).set_index('file')

In [8]:
def get_conv_df(file:str):
    _, date, group = file.split('.')[0].split('-')
    eaf = elan.read_eaf(os.path.join(transcr_loc, file))
    dial = pd.DataFrame(eaf.to_csv_rows(), columns=['speaker', '?', 'start','stop','duration','text'])
    dial.start = dial.start.astype(float)
    dial.stop = dial.stop.astype(float)
    dial = dial[~dial.text.isin(['#', ''])].sort_values('start').reset_index(drop=True)
    dial.drop(columns=['?'], inplace=True)
    dial.speaker = dial.speaker.apply(lambda x: x.split('-')[-1])
    mark = markers.loc[f'{date}_{group}']
    fconv = dial[dial.start >= (mark['End Task 1'] + 2)] # giving seconds to start task
    return dial, fconv

def ann_overlap(df):
    # df if ordered by start, stop. so the line before (more before if several during one ipu) should tell if 
    df['overlap'] = False
    for idx, row in df.iterrows():
        tmp = df.loc[(row.speaker != df.speaker) & (df.start <= row.start) & (df.stop >= row.stop)]
        if tmp.shape[0] > 0:
            df.loc[idx, 'overlap'] = True

def is_feedback(s:str, threshold:float=.5) -> bool:
    words = ['ouais','mh','hm', "c'est sûr", 'oui', 'ha','ah', '@', 'alors', 'voilà', 'ok', 'du coup']
    nw = re.findall(re.compile('('+'|'.join(words)+')'), s.lower())
    return (len(nw) / len(s.split())) > threshold

In [9]:
def create_texts(df:pd.DataFrame, max_nb_tokens:int=2048, idx_add_after_breakpoint:int=1):
    """Max number of tokens for API: 2048/4000 - cannot use overlap since anwsers might not overlap"""
    memory = []
    # 1. get the number of tokens for every line in df
    ### Which is the best option so as not to get almost empty last line?
    df['text_updated'] = df.apply(lambda x: f"<{x.speaker}> {x.text}", axis=1)
    #df['speaker_updated'] = (df.speaker != df.speaker.shift())*df.speaker
    #df['text_updated'] = df.apply(lambda x: f"<{x.speaker_updated}> {x.text}" if x.speaker_updated != '' else x.text, axis=1)
    df['nb_tok'] = df.text_updated.apply(get_number_tokens)
    df['nb_tok_cs'] = df.nb_tok.cumsum()
    # 2. while df is not empty, cut, add (start_index, text) to memory, and loop
    nb_splits = np.ceil( df.nb_tok.cumsum().iloc[-1] / max_nb_tokens ) 
    max_nb_tokens = df.nb_tok.cumsum().iloc[-1] // nb_splits
    print(f"Max number of tokens (speakers everywhere): {df.nb_tok.cumsum().iloc[-1]}; Number of splits: {int(nb_splits)}, Max number of tokens: {max_nb_tokens}")
    while df.shape[0] > 0:
        idx_change = df[df['nb_tok_cs'] <= max_nb_tokens].index[-1] + idx_add_after_breakpoint # +1 or +5
        #t = df[df['nb_tok_cs'] <= max_nb_tokens] # include the next one too
        t = df.loc[:idx_change]
        t['speaker_updated'] = (t.speaker != t.speaker.shift())*t.speaker
        t['text_updated'] = t.apply(lambda x: f"<{x.speaker_updated}> {x.text}" if x.speaker_updated != '' else x.text, axis=1)
        d = {'idx_start': t.index[0], 'idx_stop': t.index[-1], 'text': ' '.join(t.text_updated.tolist())}
        d['len'] = get_number_tokens(d['text'])
        memory.append(d)
        # update df
        #df = df[df['nb_tok_cs'] >= max_nb_tokens]
        df = df.loc[(idx_change+1):]
        df['nb_tok_cs'] = df.nb_tok.cumsum()
    # 3. return memory
    return memory

In [10]:
def locate_read_dilemma(s:str):
    dilemma = "trois passagers sont dans une montgolfière qui perd rapidement de l'altitude et qui est en passe de s'écraser le seul moyen pour les passagers de survivre est de sacrifier l'un d'eux parmi ces passagers se trouvent un scientifique dont les recherches pourraient amener au développement d'une thérapie révolutionnaire contre le cancer une maîtresse d'école primaire enceinte et son mari qui est également le pilote de la montgolfière quelle serait la meilleure solution à leur proposer"
    ld = len(dilemma.split())
    ss = s.split()
    ls = len(ss)
    # attempt match
    sm = SequenceMatcher()
    sm.set_seq1(dilemma)
    res = []
    for i in range(0, ls - ld, 5):
        x = ' '.join(ss[i:i+ld+5])
        sm.set_seq2(x)
        res.append( {'idx': i, 'lidx': i+ld+5, 'sent': x, 'ratio': sm.ratio()} )
    res = pd.DataFrame(res).sort_values('ratio', ascending=False).head(3)
    # analyse
    if res.ratio.mean() > 0.5:
        return int(res.idx.min())
    else:
        return 0

### Parse response and match to text

In [13]:
def parse_chatgpt_res(text:str, mode:str='quotes_needed') -> int: # use with literal_eval
    """Match themes, quotes based on pattern: [1./ -] (theme)(:|\( eventual sentence) "<speaker> quote"
    """
    auth_modes = ['quotes_needed','all']
    if mode not in auth_modes:
        raise ValueError(f"Argument mode must be one of {auth_modes}")
    p = re.compile(r"""
^ # Beginning of line
#(?:\d+\.|-)[ ] # Bullet or numbered point
[tT]hème[ ](?:\d+:)[ ]
(?P<theme>[^(:\n]+) # Theme of the conversation (any character that isn't `:` or `(`)
(
  (?::[ ]|[ ]\(|\n|\n[ ]) # Beginning of context (match `: ` or ` (`)
  (?P<context>
    .*?[ ]? # Context is anything, maybe followed by a space
    (?:"(?P<quote>(.*?))")? # Quote must be between `"`
  )
  \)?[ ]?
)?
\.?
$ # End of line
""", re.VERBOSE | re.MULTILINE)

    themes = []
    quotes = []
    for m in p.finditer(text):
        theme = m.group("theme")
        quote = m.group("quote")
        if mode == 'quotes_needed':
            if quote is not None:
                themes.append((theme, quote))
        else:
            themes.append(theme)
            quotes.append(quote)
    if (len(quotes) == 0) or all([q is None for q in quotes]): quotes = None
    return themes, quotes

def clean_sent(s):
    # select sentence only - form <speaker> (need to check if accurate) ... [text] ...
    if s[0] == "<" and s[3] == ">":
        speaker = s[1:3]
        s = s[5:]
    else: speaker = None
    if s[0:3] == "...":
        s = s[4:]
    if s[-3:] == "...":
        s = s[:-3]
    return s, speaker

def match_sentence_loc(s, mem, df):
    s, speaker = clean_sent(s) #literal_eval(s)) needed for older version
    # check if matching original sentences
    #if s in mem['text']:
    # method 1: match (consecutive) parts of the sentence to the extract - issue: sole (common) words matching the extract
    t = df.loc[mem['idx_start']:(mem['idx_stop']+1)]
    if speaker is not None:
        t = t[t.speaker == speaker]
    t['ratio'] = t.text.apply(lambda x: len(x) if x in s else 0)
    sent_start = t.groupby([((t.ratio > 0) != (t.ratio > 0).shift()).cumsum()]).agg({'start': 'min', 'ratio': 'sum'}).reset_index(drop=True).sort_values('ratio', ascending=False).iloc[0]
    # issue: partial sentence => fall back to difflib
    if sent_start.ratio > 5: # usually long sentences
        return sent_start.start
    # method 2: use difflib.SequenceMatcher() .set_seq1(s) df.apply(sm.set_seq2(x); sm.ratio()) but issue if the sentence is split()
    sm = SequenceMatcher()
    sm.set_seq1(s)

    def apply_seqmatch(x):
        sm.set_seq2(x)
        return sm.ratio()
    t['ratio'] = t.text.apply(apply_seqmatch)
    sent_start = t.sort_values('ratio', ascending=False).iloc[0]
    return sent_start.start

def match_themes_loc(res:str, mem, df, theme_col:str='theme'):
    themes, quotes = parse_chatgpt_res(res)
    if quotes is None:
        for theme, sent in themes:
            sent_start = match_sentence_loc(sent, mem, df)
            df.loc[df.start == sent_start, theme_col] = theme
    else:
        for i, (t, sent) in enumerate(zip(themes, quotes)):
            if sent is None:
                continue
            sent_start = match_sentence_loc(sent, mem, df)
            t = f"theme{i}" if t is None else t
            df.loc[df.start == sent_start, theme_col] = t

#### Old

In [12]:
def parse_chatgpt_res(s:str): # use with literal_eval
    """ OLD, updated format :/
    Format: 
    ----------
    ```
    Thèmes de la conversation:
    - [liste ]
    
    Phrases introduisant les thèmes:
    ```

    Difficultés:
    ----------
    * Nombre différent de thèmes et de phrases (dans ce cas les quotes ne sont pas associées à un thème)
    * Variations dans la formulation précise des intitulés (`introduisant chaque thème`, `introduisant ces thèmes`)
    * Phrases des quotes coupées ==> comment matcher ?
    """
    themes, quotes = s.split('\n\n')
    themes = themes.split('\n- ')[1:]
    quotes = quotes.split('\n- ')[1:]
    if (len(themes) == len(quotes)) and len(quotes[0].split(':')) >= 2:
        return [(t, ': '.join(q.split(': ')[1:])) for t, q in zip(themes, quotes)], None
    else:
        print("Skipping answser, different number of themes and quotes")
        return themes, quotes

## Calls

### Random tests

In [19]:
s = "Who won the world series in 2020?"
call_api(s)

('The Los Angeles Dodgers won the World Series in 2020.', 31)

### Apply on one file

In [ ]:
res = []
for file in transcr_files:
    _, fconv = get_conv_df(file)
    ann_overlap(fconv)
    fconv['is_feedback'] = fconv.overlap + fconv.text.apply(is_feedback)
    fconv = fconv[~fconv.is_feedback].reset_index(drop=True)
    mem = ' '.join(fconv.text.tolist())
    res = (locate_read_dilemma(mem))
    #res.append({'file': file, 'idx': mem.idx.tolist(), 'lidx': mem.lidx.max(), 'ratio': mem.ratio.mean(), 
    #            'std': mem.ratio.std(), 'end': mem.sent.iloc[0][-40:]})
    print(file, ' '.join(mem.split()[res:res+30]))
#res = pd.DataFrame(res)

In [153]:
file_selector = widgets.Dropdown(options = transcr_files, description="Which file?")
file_selector

Dropdown(description='Which file?', options=('bkt-221116-CGLS.eaf', 'bkt-221117-TFGG.eaf', 'bkt-221118-GDNF.ea…

In [239]:
file = file_selector.value
dial, fconv = get_conv_df(file)

ann_overlap(fconv)
fconv['is_feedback'] = fconv.overlap + fconv.text.apply(is_feedback)
fconv = fconv[~fconv.is_feedback].reset_index(drop=True)

In [218]:
#res, nb_tok = call_api(t)
#res, res2, nb_tok, nb_tok2 # print(res)

In [240]:
mem = create_texts(fconv.copy(deep=True), idx_add_after_breakpoint=5)
l_speakers = fconv.speaker.unique().tolist()

Max number of tokens (speakers everywhere): 3758; Number of splits: 2, Max number of tokens: 1879.0


In [241]:
calls = {}
for i in tqdm(range(len(mem))):
    p = create_prompt(mem[i]['text'], speakers = l_speakers)
    res, nb_tok = call_api(p)
    # currently res, res2
    calls[i] = {'split': mem[i], 'prompt': p, 'response': res, 'nb_tokens': nb_tok}
    time.sleep(10)

100%|██████████| 2/2 [00:32<00:00, 16.20s/it]


In [242]:
print(calls[0]['response'])

thème 1: Situation d'urgence (phrase d'introduction: "trois passagers sont dans une mongolfière qui perd rapidement de latitude et qui est en passe de s'écraser...")
thème 2: Dilemme moral (phrase d'introduction: "il y en a un qui doit se sacrifier du coup")
thème 3: Possibilité de survie (phrase d'introduction: "j'aurais peut-être essayé de jeter des sacs deux d'abord très et qui l'ont déjà fait en la fin ce serait la meilleure solution à proposer")
thème 4: Valeurs et morale (phrase d'introduction: "quand on juge les valeurs d'une ville à clermont")
thème 5: Choix de la victime (phrase d'introduction: "dans tous les cas je ne serai pas sa je suis la maîtresse d'école eh ben elle a une vie dans son ventre je ne sais pa")
thème 6: Possibilité de volontariat (phrase d'introduction: "tout ça dépend de moi pourquoi qui je demanderais qui est volontaire d'abord")
thème 7: Alternative au sacrifice (phrase d'introduction: "ou alors il n'y en a aucun qui ce soir ils décident de mourir tous le

**Saving responses**

In [243]:
mem_path = os.path.expanduser('~/Downloads/chatgpt-calls.json')
if os.path.exists(mem_path):
    with open(mem_path, 'r') as f:
        mem_global = json.load(f)
else:
    mem_global = {}

mem_global[file] = calls
with open(mem_path, 'w') as f:
    json.dump(mem_global, f, indent=4)

**Parse responses**

In [244]:
parse_chatgpt_res(calls[0]['response'], 'all')

(["Situation d'urgence",
  'Dilemme moral',
  'Possibilité de survie',
  'Valeurs et morale',
  'Choix de la victime',
  'Possibilité de volontariat',
  'Alternative au sacrifice'],
 ["trois passagers sont dans une mongolfière qui perd rapidement de latitude et qui est en passe de s'écraser...",
  'il y en a un qui doit se sacrifier du coup',
  "j'aurais peut-être essayé de jeter des sacs deux d'abord très et qui l'ont déjà fait en la fin ce serait la meilleure solution à proposer",
  "quand on juge les valeurs d'une ville à clermont",
  "dans tous les cas je ne serai pas sa je suis la maîtresse d'école eh ben elle a une vie dans son ventre je ne sais pa",
  "tout ça dépend de moi pourquoi qui je demanderais qui est volontaire d'abord",
  "ou alors il n'y en a aucun qui ce soir ils décident de mourir tous les trois"])

In [245]:
fconv['theme'] = None
for i in range(len(calls)):
    res = calls[i]['response']
    memo = calls[i]['split']
    match_themes_loc(res, memo, fconv)
fconv['theme_filled'] = fconv.theme.ffill()

**Saving DF**

In [246]:
csv_path = os.path.expanduser('~/Downloads/chatgpt-themes-brainkt.csv')
fcols = fconv.columns.tolist()
fpattern = file.replace('-','_')[5:-4]
fconv['file'] = fpattern
fconv = fconv[['file']+fcols] # reordering columns
if os.path.exists(csv_path):
    fdf = pd.read_csv(csv_path, na_values=[''])
    if fpattern in fdf.file.unique():
        fdf = fdf[fdf.file != fpattern]
    fdf = pd.concat([fdf, fconv], axis=0).reset_index(drop=True)
else:
    fdf = fconv
fdf.to_csv(csv_path, index=False)

**For previous files update:**

In [247]:
mf = sorted(list(set([f.replace('-','_')[5:-4] for f in transcr_files]) - set(fdf.file.unique().tolist())))
print(f"Missing files: {len(mf)}")
print(mf)

Missing files: 0
[]


In [227]:
redo_files = list(mem_global.keys())
frev_selector = widgets.Dropdown(options = redo_files, description="Which file?")
frev_selector

Dropdown(description='Which file?', options=('bkt-221116-CGLS.eaf', 'bkt-221117-TFGG.eaf', 'bkt-221118-VPET.ea…

In [284]:
file = frev_selector.value
calls = mem_global[file]
calls = {int(k):v for k,v in calls.items()}
# recompute fconv
dial, fconv = get_conv_df(file)
ann_overlap(fconv)
fconv['is_feedback'] = fconv.overlap + fconv.text.apply(is_feedback)
fconv = fconv[~fconv.is_feedback].reset_index(drop=True)

### Analysis
* Number of themes / file
* Number of manual themes (aggregated)
* Duration (ipus/time) of dilemma / themes

In [ ]:
# printing for manual review
# for each file - grab from mem_global[file]['response'] the themes (esp if some are not annotated)
# for each file - groupby shift().cumsum() => join list => 
for file in mem_global.keys():
    file_pattern = file.replace('-','_')[5:-4]
    if file_pattern in fdf.file.unique():
        print("\n\n", file, end='\n---------------\n')
        print('\n'.join(mem_global[file][x]['response'] for x in sorted(mem_global[file])))
        t = fdf[fdf.file == file_pattern]
        t['speaker_updated'] = (t.speaker != t.speaker.shift())*t.speaker
        t['text_updated'] = t.apply(lambda x: f"<{x.speaker_updated}> {x.text}" if x.speaker_updated != '' else x.text, axis=1)
        tg = t.groupby((t['theme_filled'].fillna('None') != t['theme_filled'].fillna('None').shift()).cumsum()).agg(
            {'theme_filled':lambda x: list(x)[0], 'text_updated': lambda x: ' '.join(list(x))}).set_index('theme_filled')
        for theme, row in tg.iterrows():
            print('-------', theme, '-------')
            print(row.text_updated)

In [270]:
theme_col = 'theme_filled'
fdf_an = []
for idx, group in fdf.groupby(['file', (fdf[theme_col].fillna('None') != fdf[theme_col].fillna('None').shift()).cumsum()]):
    spk_1 = idx[0][-4:-2]
    d = {
        'file': idx[0], 'theme_idx': idx[1], 'theme': group[theme_col].iloc[0], 'nb_ipus': group.shape[0], 
        'start': group.start.iloc[0], 'stop': group.stop.iloc[-1], 'duration': group.stop.iloc[-1] - group.start.iloc[0],
        'first_spk': f'spk{2-(group.speaker.iloc[0] == spk_1)}',
        'dur_spk1': group[group.speaker == spk_1].duration.astype(float).sum(), 
        'dur_spk2': group[group.speaker != spk_1].duration.astype(float).sum(), 
    }
    for i in [1,2]: 
        d[f'ratio_spk{i}'] = d[f'dur_spk{i}'] / (d['dur_spk1']+d['dur_spk2'])
    fdf_an.append(d)
fdf_an = pd.DataFrame(fdf_an)
fdf_an.head(2)

,file,theme_idx,theme,nb_ipus,start,stop,duration,first_spk,dur_spk1,dur_spk2,ratio_spk1,ratio_spk2
0,21116_CGLS,1,NaN,100,1018.305,1343.192,324.887,spk1,99.385,113.129,0.467663,0.532337
1,21116_CGLS,2,Choix de la personne à jeter d'une montgolfière,51,1346.085,1491.300,145.215,spk2,50.361,55.848,0.474169,0.525831


In [273]:
fdf_anf = fdf_an.groupby('file').agg({'theme_idx': lambda x: len(list(x)), 'duration': 'mean', 'dur_spk1': 'sum', 'dur_spk2': 'sum'})
for i in [1,2]:
    fdf_anf.loc[:,f'ratio_spk{i}'] = fdf_anf[f'dur_spk{i}']/(fdf_anf.dur_spk1+fdf_anf.dur_spk2)
fdf_anf.head(2)

,theme_idx,duration,dur_spk1,dur_spk2,ratio_spk1,ratio_spk2
file,,,,,,
21116_CGLS,8,108.454625,266.406,388.403,0.406845,0.593155
21117_TFGG,12,74.090667,139.584,362.688,0.277905,0.722095


## Notes
**Difficulties**: various ways to answer depending on days/number of calls?

23/04/25:
```
-------- prompt 1
Thèmes de la conversation: 
[4 themes]

Phrases introduisant ces thèmes:
[6 sentences]

-------- prompt 2
Thèmes de la conversation: 
- Choix difficile à faire dans une situation hypothétique (sauter d'une montgolfière)
- Orientation professionnelle et projets d'avenir
- Sujet de recherche en doctorat et expériences à mettre en place

Phrases introduisant chaque thème:
- Choix difficile: "<CG> Après s'ils sont pas très haut, qu'il y en a un qui est un petit peu adroit de ses jambes, il saute et il se réceptionne..."
- Orientation professionnelle: "<CG> Et du coup, tu fais quoi, toi? T'es en... en licence? <LS> Alors euh... j'ai une licence en management et je me réoriente, j'avais commencé un master en achat en école de commerce et je me réoriente dans le sport et euh... j'ai passé de concours pour être moniteur de sport dans l'armée de l'air..."
- Sujet de recherche: "<CG> euh... Mon sujet c'est euh... la... l'adaptation du langage à l'adolescence..."
```

23/04/26:
```
-------- prompt 1
Thèmes de la conversation :
1. Le dilemme de la mongolfière qui perd de l'altitude (début de la conversation avec la lecture du dilemme)
2. Les différentes solutions envisageables pour résoudre le dilemme (proposition d'essayer de poser la mongolfière, balance pour peser les passagers, etc.)
3. Le choix difficile à faire entre la vie du scientifique et celle du couple (discussion sur l'utilité de chaque passager, proposition de sacrifier personne, etc.)
4. La réflexion sur la situation elle-même (contexte, possibilité d'autres solutions, etc.)

-------- prompt 2
Les différents thèmes de la conversation sont :

- Choix de la personne à jeter d'une montgolfière (débutant avec la phrase "<LS> c'est un scientifique merde").
- Orientation professionnelle et études, expériences professionnelles passées (débutant avec la phrase "<CG> Et du coup, tu fais quoi, toi? T'es en... en licence?").
- Sujet de recherche en doctorat et expériences à mettre en place (débutant avec la phrase "<CG> euh... Mon sujet c'est euh... la... l'adaptation du langage à l'adolescence").
```

Variations in: the way to deliver the information. However, the _content_ hasn't changed much: same number of themes (and inability to give sentences) for prompt1, 3 themes with about the same sentences for prompt2

## Comparison with other methods: TextTiling

In [113]:
from nltk.tokenize import TextTilingTokenizer

In [117]:
def one_tt(s:str, ttt = TextTilingTokenizer()):
    """Analyse a conversation using TextTiling to generate topics segmentation.
    Then retrace to initial conversation using '\n' as utterance breaks.
    Returns a dict and a list: the dict contains a list of utterances associated with each topic, 
    the list containing the index of the topic for each utterance (shape: df[df.file == X].shape[0])
    """
    d_utter = {}
    l_idx = []
    tokens = ttt.tokenize(s)
    for i, token in enumerate(tokens):
        tok_utter = token.split("\n\n")
        tok_utter = [x for x in tok_utter if x != ''] # remove ' from list
        d_utter[i] = tok_utter
        l_idx += [i]*len(tok_utter)

    return d_utter, l_idx

d_utter, l_idx = one_tt('\n\n'.join(fconv.text.tolist()))
assert(fconv.shape[0] == len(l_idx))
fconv['TextTiling'] = l_idx

In [122]:
fconv.columns

Index(['speaker', 'start', 'stop', 'duration', 'text', 'overlap',
       'is_feedback', 'TextTiling'],
      dtype='object')

In [ ]:
for _, group in fconv.groupby('TextTiling'):
    group = group[~group.is_feedback]
    group['speaker_updated'] = (group.speaker != group.speaker.shift())*group.speaker
    group['text_updated'] = group.apply(lambda x: f"<{x.speaker_updated}> {x.text}" if x.speaker_updated != '' else x.text, axis=1)
    print(' '.join(group.text_updated.tolist()))
# very short themes, chatgpt are longer (slightly)

## Adding themes annotation to eaf files